<a href="https://colab.research.google.com/github/jxxxuan/my_res/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import  tensorflow as tf
from    tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from 	tensorflow import keras
import  os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


def preprocess(x, y):
    # [0~255] => [-1~1]
    x = 2 * tf.cast(x, dtype=tf.float32) / 255. - 1.
    y = tf.cast(y, dtype=tf.int32)
    return x,y


batchsz = 128
# [50k, 32, 32, 3], [10k, 1]
(x, y), (x_val, y_val) = datasets.cifar10.load_data()
y = tf.squeeze(y)
y_val = tf.squeeze(y_val)
y = tf.one_hot(y, depth=10) # [50k, 10]
y_val = tf.one_hot(y_val, depth=10) # [10k, 10]
print('datasets:', x.shape, y.shape, x_val.shape, y_val.shape, x.min(), x.max())


train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.map(preprocess).shuffle(10000).batch(batchsz)
test_db = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_db = test_db.map(preprocess).batch(batchsz)


sample = next(iter(train_db))
print('batch:', sample[0].shape, sample[1].shape)


class MyDense(layers.Layer):
    # to replace standard layers.Dense()
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()

        self.kernel = self.add_variable('w', [inp_dim, outp_dim])
        # self.bias = self.add_variable('b', [outp_dim])

    def call(self, inputs, training=None):

        x = inputs @ self.kernel
        return x

class MyNetwork(keras.Model):

    def __init__(self):
        super(MyNetwork, self).__init__()

        self.fc1 = MyDense(32*32*3, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)



    def call(self, inputs, training=None):
        """

        :param inputs: [b, 32, 32, 3]
        :param training:
        :return:
        """
        x = tf.reshape(inputs, [-1, 32*32*3])
        # [b, 32*32*3] => [b, 256]
        x = self.fc1(x)
        x = tf.nn.relu(x)
        # [b, 256] => [b, 128]
        x = self.fc2(x)
        x = tf.nn.relu(x)
        # [b, 128] => [b, 64]
        x = self.fc3(x)
        x = tf.nn.relu(x)
        # [b, 64] => [b, 32]
        x = self.fc4(x)
        x = tf.nn.relu(x)
        # [b, 32] => [b, 10]
        x = self.fc5(x)

        return x


network = MyNetwork()
network.compile(optimizer=optimizers.Adam(lr=1e-3),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
network.fit(train_db, epochs=15, validation_data=test_db, validation_freq=1)

network.evaluate(test_db)
network.save_weights('ckpt/weights.ckpt')
del network
print('saved to ckpt/weights.ckpt')


network = MyNetwork()
network.compile(optimizer=optimizers.Adam(lr=1e-3),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
network.load_weights('ckpt/weights.ckpt')
print('loaded weights from file.')
network.evaluate(test_db)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers,layers,datasets

#the most frequest words
total_words = 10000
max_review_len = 80
batchsz = 128
embedding_len = 100
(x_train,y_train),(x_test,y_test) = datasets.imdb.load_data(num_words=total_words)
x_train = keras.preprocessing.sequence.pad_sequences(x_train,max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test,max_review_len)

db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1000).batch(batchsz,drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(batchsz,drop_remainder=True)

sample = next(iter(db_train))
print(sample[0].shape,sample[1].shape)
print(tf.reduce_max(y_train),tf.reduce_min(y_train))


class MyRNN(keras.Model):

  def __init__(self,units):
    super(MyRNN,self).__init__()
    #transform text to embedding representation
    #[b,100] => [b,80,100]
    self.embedding = layers.embedding(total_words,embedding_len,input_len=max_review_len)

    #[b,64]
    self.state0 = [tf.zeros([batchsz,units])]

    #[b,64,100], h_dim:64
    #RNN：cell1, cell2, cell3
    #SimpleRNN
    self.rnn_cell0 = layers.SimpleRNNCell(units,dropuout=0.2)

    #fc
    self.outlayer = layers.Dence(1)

  def call(self,input,traning=None):
    #[b,80]
    x = input
    #embedding:[b,80] => [b,80,100]
    x = self.embedding(x)
    #RNNcell compute
    #[b,80,100] => [b,64]
    state = self.state
    for word in tf.unstack(x,axis=1): # word: [b,100]
      (out,state1) = self.rnn_cell0(word)


17473536/17464789 [==============================] - 0s 0us/step
(128, 80) (128,)


AttributeError: ignored

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,datasets

class BasicBlock(layers.layer):
  def __init__(self,filter_num,stride=1):
    super(BasicBlock,self).__init__()
    self.conv1 = layers.conv(filter_num,(3,3),stride=stride,padding='same')
    self.bn1 = layers.BatchNormalization()
    self.relu = layers.Activation('relu')

    self.conv1 = layers.conv(filter_num,(3,3),stride=1,padding='same')
    self.bn1 = layers.BatchNormalization()

SyntaxError: ignored

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,optimizers,layers,Sequential

tf.random.set_seed(2345)

conv_layers=[
             layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),

             layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),

             layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),

             layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),

             layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
             layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same')

]
def pre_data(x,y):
  x = tf.cast(x,dtype=tf.float32) / 255.
  y = tf.cast(y,dtype=tf.int32)
  return x,y

(x,y),(x_test,y_test) = datasets.cifar100.load_data()
y = tf.squeeze(y,axis=1)
y_test = tf.squeeze(y_test,axis=1)
train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).map(pre_data).batch(64)

test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.map(pre_data).batch(64)

def main():
  

  conv_net = Sequential(conv_layers)
  
  '''
  x = tf.random.normal([4,32,32,3])
  out = conv_net(x)
  '''
  fc_net = Sequential([
              layers.Dense(256,activation=tf.nn.relu),
              layers.Dense(128,activation=tf.nn.relu),
              layers.Dense(100,activation=None)])
  
  conv_net.build(input_shape=[None,32,32,3])
  fc_net.build(input_shape=[None,512])
  optimizer = optimizers.Adam(lr=1e-4)

  variables = conv_net.trainable_variables + fc_net.trainable_variables
  for epoch in range(10):

    for step,(x,y) in enumerate(train_db):
      with tf.GradientTape() as tape:
        out = conv_net(x)
        out = tf.reshape(out,[-1,512])
        logits = fc_net(out)
        y_onehot = tf.one_hot(y,depth=100)
        loss = tf.losses.categorical_crossentropy(y_onehot,logits,from_logits=True)
        loss = tf.reduce_mean(loss)

      grads = tape.gradient(loss,variables)
      optimizer.apply_gradients(zip(grads,variables))

      if step%100 == 0:
        print(epoch,step,'loss:',loss.numpy())

    total_num = 0
    total_correct = 0
    for x,y in test_db:
      out = conv_net(x)
      out = tf.reshape(out,[-1,512])
      logits = fc_net(out)
      prob = tf.nn.softmax(logits,axis=1)
      pred = tf.argmax(prob, axis=1)
      pred = tf.cast(pred, dtype=tf.int32)

      correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
      correct = tf.reduce_sum(correct)

      total_num += x.shape[0]
      total_correct += int(correct)

    acc = total_correct / total_num
    print(epoch, 'acc:', acc)

if __name__ == '__main__':
  main()


In [ ]:
!nvidia-smi

Sun Apr 17 14:04:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    59W / 149W |   4377MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------